# **VET-ASSISTANT LLM**

**Installing dependencies**

In [ ]:
!pip install gradientai --upgrade
!pip install langchain
!pip install -U gradient_haystack
!pip install regex
!pip install rouge
!pip install nltk
!pip install wandb
!pip install datasets
!pip install rouge
!pip install nltk
!pip install langchain_community


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.1 MB/s eta 0:00:00


## **1.Loading the Dataset**

In [ ]:
import json
import requests

# URL of the online repository where the dataset is hosted
dataset_url = "https://raw.githubusercontent.com/swafey-karanja/Model-training/main/NewData.json"

# Make an HTTP GET request to fetch the dataset
response = requests.get(dataset_url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Load the dataset from the response content
    train_dataset = json.loads(response.text)

    # Print the size of the dataset
    print("Dataset Size:", len(train_dataset))
else:
    # Print an error message if the request failed
    print("Failed to fetch dataset. Status code:", response.status_code)


Dataset Size: 1412


**Break the data into batches**

In [ ]:
def divide_into_Batches(number, chunk_size):  # Define a function to divide a number into chunks of a given size
    Batches = []
    while number > 0:
        if number >= chunk_size:
            Batches.append(chunk_size)
            number -= chunk_size
        else:
            Batches.append(number)
            break

    return Batches

Batches = divide_into_Batches(len(train_dataset), 100)  # Divide the dataset into chunks of 100 samples each
print("Batches size")
print(Batches)

Batches size
[100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 12]


## **2.Load the Base Model.**

Loading the Nous-Hermes-Llama2-13b base model

In [ ]:
import os
os.environ['GRADIENT_ACCESS_TOKEN'] = "Y2XG6OMr6oZbZETWKkPKogmWshFlD5ft"
os.environ['GRADIENT_WORKSPACE_ID'] = "c571b959-4ce8-474a-b3f3-398c7b347c57_workspace"

In [ ]:
import json
import os
from gradientai import Gradient
from langchain.chains import LLMChain
from langchain.llms import GradientLLM
from langchain.prompts import PromptTemplate


os.environ['GRADIENT_ACCESS_TOKEN'] = "Y2XG6OMr6oZbZETWKkPKogmWshFlD5ft"
os.environ['GRADIENT_WORKSPACE_ID'] = "c571b959-4ce8-474a-b3f3-398c7b347c57_workspace"

gradient =  Gradient()

print("Available Base Models")
for i in gradient.list_models(only_base=True):
    print("\t",i)

base_model_id = "NousResearch/Nous-Hermes-Llama2-13b"
base_model_name = "nous-hermes2"
base_model = gradient.get_base_model(base_model_slug="nous-hermes2") # base model Nous-Hermes-Llama2-13b

print("\nBase Model Chosen :", base_model)

Available Base Models

Base Model Chosen : <gradientai._base_model.BaseModel object at 0x7b7224a40d60>


##**3. Creating a Model Adapter**

* Adapters are small, lightweight modules inserted between
existing layers of a pre-trained LLM. They act like "add-ons" that focus on learning task-specific information without modifying the core knowledge captured in the original model.

* The addapter server as the object that we are going to fine tune

In [ ]:
our_finetune_model_name="Llama2-13b/Vet-assistant-Newdata"
Fine_Tune__adapter = base_model.create_model_adapter(
        name=our_finetune_model_name, # base mode nous hermis
        learning_rate=0.00005, #  Determines how fast a model updates its knowledge during fine-tuning.
        rank=8,  # Dimensionality

    )

# default hyperparameters Frozen
hyperparameters = {
                  "block_size": 1024,
                  "model_max_length": 2048,
                  "padding": "right",
                  "use_flash_attention_2": False,
                  "disable_gradient_checkpointing": False,
                  "logging_steps": -1,
                  "evaluation_strategy": "epoch",
                  "save_total_limit": 1,
                  "save_strategy": "epoch",
                  "auto_find_batch_size": False,
                  "mixed_precision": "fp16",
                  "epochs": 3,
                  "batch_size": 100,
                  "warmup_ratio": 0.1,
                  "gradient_accumulation": 1,
                  "optimizer": "adamw_torch",
                  "scheduler": "linear",
                  "weight_decay": 0,
                  "max_grad_norm": 1,
                  "seed": 42,
                  "apply_chat_template": False,
                  "quantization": "int4",
                  "target_modules": "",
                  "merge_adapter": False,
                  "peft": True,
                  "lora_r": 16,
                  "lora_alpha": 32,
                  "lora_dropout": 0.05
  }


print(f"Base model id                : {Fine_Tune__adapter._base_model_id}")
print(f"Fine tune model Name         : { Fine_Tune__adapter.name}")
print(f"Fine tune model adapter id   : {Fine_Tune__adapter.id}")

print("\n\n")
print("Size of object in memory, in bytes.", Fine_Tune__adapter.__format__.__sizeof__()) # Size of object in memory, in bytes.
Fine_Tune__adapter.__dict__

Base model id                : cc2dafce-9e6e-4a23-a918-cad6ba89e42e_base_ml_model
Fine tune model Name         : Llama2-13b/Vet-assistant-Newdata
Fine tune model adapter id   : 2e7a849b-239e-44a1-a88a-3be1d77c1f93_model_adapter



Size of object in memory, in bytes. 56


{'_api_instance': <gradientai.openapi.client.api.models_api.ModelsApi at 0x7b7224a41630>,
 '_id': '2e7a849b-239e-44a1-a88a-3be1d77c1f93_model_adapter',
 '_workspace_id': 'c571b959-4ce8-474a-b3f3-398c7b347c57_workspace',
 '_async_semaphore': <asyncio.locks.Semaphore object at 0x7b7224a42680 [unlocked, value:8]>,
 '_base_model_id': 'cc2dafce-9e6e-4a23-a918-cad6ba89e42e_base_ml_model',
 '_name': 'Llama2-13b/Vet-assistant-Newdata'}

## **4. Fine Tuning The Adaptor**

For this case we will be performing Laura-based finetuning which means that we are freezing about 99% of the layers and then finetuning an adapter on top of it.

LoRA: Low-Rank Adaptation of Large Language Models is a novel technique introduced by Microsoft researchers to deal with the problem of fine-tuning large-language models with billions of parameters

 LoRA proposes to freeze pre-trained model weights and inject trainable layers (rank-decomposition matrices) in each transformer block. This greatly reduces the number of trainable parameters and GPU memory requirements since gradients don't need to be computed for most model weights


why LoRA finetuning:

1. Faster Training: Since only the added task-specific layers are trained while the pre-trained model's parameters remain frozen, the fine-tuning process is generally faster compared to training a model from scratch
2. Computation requirements are lower. We could create a full fine-tuned model in a 2080 Ti with 11 GB of VRAM!
3. Trained weights are  much smaller. Because the original model is frozen and we inject new layers to be trained

 [for more info](https://huggingface.co/blog/lora)

In [ ]:
print(f"Our Model id:  {Fine_Tune__adapter.id}")
num_epochs = 1  # num_epochs is the number of times you fine-tune the model # more epochs tends to get better results, but you also run the risk of "overfitting"
count = 0
print("================================================================\n")
print("Fine tuning . . .\n")
while count < num_epochs:
    print(f"Fine-tuning the model, iteration {count + 1}")
    s = 0
    n = 1
    for Batch in Batches:
        print(f"Batch {n} range: {s} : {(s + Batch)}")

        # Try to fine-tune the model with the chunk of samples,
        while True:
            try:
                metric = Fine_Tune__adapter.fine_tune(samples=train_dataset[s: s + Batch])
                print(f"\t Batch {n} Evaluation :", metric)
                break
            except:
                pass



        s += Batch
        n += 1
    count = count + 1

Our Model id:  2e7a849b-239e-44a1-a88a-3be1d77c1f93_model_adapter

Fine tuning . . .

Fine-tuning the model, iteration 1
Batch 1 range: 0 : 100
	 Batch 1 Evaluation : number_of_trainable_tokens=8007 sum_loss=18044.285
Batch 2 range: 100 : 200
	 Batch 2 Evaluation : number_of_trainable_tokens=8043 sum_loss=11213.263
Batch 3 range: 200 : 300
	 Batch 3 Evaluation : number_of_trainable_tokens=9278 sum_loss=10536.057
Batch 4 range: 300 : 400
	 Batch 4 Evaluation : number_of_trainable_tokens=10454 sum_loss=11967.004
Batch 5 range: 400 : 500
	 Batch 5 Evaluation : number_of_trainable_tokens=13087 sum_loss=13200.826
Batch 6 range: 500 : 600
	 Batch 6 Evaluation : number_of_trainable_tokens=16332 sum_loss=17247.604
Batch 7 range: 600 : 700
	 Batch 7 Evaluation : number_of_trainable_tokens=18421 sum_loss=17550.602
Batch 8 range: 700 : 800
	 Batch 8 Evaluation : number_of_trainable_tokens=18931 sum_loss=17459.293
Batch 9 range: 800 : 900
	 Batch 9 Evaluation : number_of_trainable_tokens=13457 sum

## **5.Model Inference**

"model inference" typically refers to the process of using a trained model to make predictions on new, unseen data. Fine-tuning a model involves taking a pre-trained model and further training it on a specific task or dataset to improve its performance.

When fine-tuning a model, the process of inference remains the same as with any other trained model. Once the fine-tuning is complete, you can use the model to make predictions on new data by passing the data through the model and obtaining the output.

In [ ]:
from langchain.chains import LLMChain    # Import the LLMChain class for building LLM-based workflows
from langchain.llms import GradientLLM   # Import the GradientLLM class for interacting with Gradient AI's API
from langchain.prompts import PromptTemplate # Import the PromptTemplate class for defining how to prompt the LLM
import gradientai
import os # Import the os module for potential file system interactions

In [ ]:
Fine_Tune__adapter_ID = "2e7a849b-239e-44a1-a88a-3be1d77c1f93_model_adapter"
#Fine_Tune__adapter_ID = Fine_Tune__adapter.id
#  creating a GradientLLM object
llm = GradientLLM(
    model=Fine_Tune__adapter_ID,
    model_kwargs=dict(
        max_generated_token_count=128, # Adjust how your model generates completions
        temperature = 0.7, # randomness
        top_k=50 # Restricts the model to pick from k most likely words,
    ),
)

### **Formatting Prompts**

The model follows the Alpaca prompt format which provides a structured way to input information to the model, guiding it on what kind of output is desired.

In [ ]:
template = """### Instruction: {Instruction} \n\n### Response:"""

prompt = PromptTemplate(template=template, input_variables=["Instruction"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


#### *Example* *inputs* *and* *corresponding* *outputs*

In [ ]:
Question1 = "How does Clostridial Diseases impact milk production, reproductive performance, and overall well-being in cows?"
Question2 = "what are some of the risk factors associated with lameness in dairy cows?"
Question3 = "How dose lameness impact mill production, reproduction and overall well-being in cows"
Question4 = "What diseases are prevelant in dairy small ruminant, and what managment practice can mitigate their impact "
Question5 = "what specific health managment strategies should be implemented to prevent or treat common cow diseases?"


print("Question :\n", Question1)
Answer = llm_chain.run(Instruction=f"{Question1}")
print("Answer :\n", Answer, "\n\n")

print("Question :\n", Question2)
Answer = llm_chain.run(Instruction=f"{Question2}")
print("Answer :\n", Answer, "\n\n")

print("Question :\n", Question3)
Answer = llm_chain.run(Instruction=f"{Question3}")
print("Answer :\n", Answer, "\n\n")

print("Question :\n", Question4)
Answer = llm_chain.run(Instruction=f"{Question4}")
print("Answer :\n", Answer, "\n\n")

print("Question :\n", Question5)
Answer = llm_chain.run(Instruction=f"{Question5}")
print("Answer :\n", Answer, "\n\n")


Question :
 How does Clostridial Diseases impact milk production, reproductive performance, and overall well-being in cows?


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Answer :
 Clostridial Diseases can cause reduced milk production, impaired reproductive performance, and decreased overall well-being in cows. 


Question :
 what are some of the risk factors associated with lameness in dairy cows?
Answer :
 Risk factors associated with lameness in dairy cows include digital dermatitis, foot rot, and claw diseases. 


Question :
 How dose lameness impact mill production, reproduction and overall well-being in cows
Answer :
 Lameness impacts mill production, reproduction, and overall well-being in cows, causing reduced milk production, decreased reproductive success, and decreased body condition. 


Question :
 What diseases are prevelant in dairy small ruminant, and what managment practice can mitigate their impact 
Answer :
 Diseases prevalent in dairy small ruminant include mastitis, foot rot, and scabies. Management practices such as regular hoof trimming, maintaining clean and dry living conditions, and treating infected animals can mitigate their 

## **6.Model Evaluation**

Here we are using two popular automatic evaluation metrics to assess the performance of your LLM:

* BLEU score: This metric calculates the n-gram precision between the generated response and the reference response

* ROUGE score: This metric measures the overlap in word n-grams and longest common subsequences between the generated response and the reference response.

BLEU and ROUGE scores are calculated to compare the generated response with the target response.

In [ ]:
import re
import json
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge
from langchain.chains import LLMChain
from langchain.llms import GradientLLM
from langchain.prompts import PromptTemplate
import os

In [ ]:


def compute_rouge_scores(hypotheses, references):
    rouge = Rouge()
    scores = rouge.get_scores(hypotheses, references, avg=True)
    return scores


def compute_bleu_score(target_response, llm_responses):
    bleu_score = corpus_bleu([target_response.split()], [llm_responses.split()])  # Calculate BLEU score
    return bleu_score


def Find_Instruction(input_pattern, input_string):
    matches = re.findall(input_pattern, input_string, re.DOTALL)

    # If there are matches, extract the first one
    extracted_string = None
    if matches:
        extracted_string = matches[0]

    return extracted_string


def Evaluate(Sample=None, count=0):
    print("\n =================================== Evaluation =================================== ")
    input_pattern = r'<s>### Instruction:\n(.*?) \n'
    response_pattern = r'Response:\n(.*?)</s>'
    bleu_scoreS = []
    rouge_scoreS = []

    if count != 0:
        iteration = count - 1
    else:
        iteration = count

    while iteration >= 0:

        input_query = Find_Instruction(input_pattern, Sample[iteration]["inputs"])
        target_response = Find_Instruction(response_pattern, Sample[iteration]["inputs"])

        if input_query and target_response is not None:
            print("\n ---------------------------------------------------------------")
            print("INPUT QUERY:\n", input_query)
            print("\nTARGET RESPONSE:\n", target_response)

            llm_responses = llm_chain.run(Instruction=f"{input_query}")
            print("\nLLM RESPONSE:\n", llm_responses)

            rouge_scores = compute_rouge_scores(llm_responses, target_response)

            bleu_score = compute_bleu_score(target_response, llm_responses)
            print("\nBLEU Score:", bleu_score)
            print("ROUGE Scores:")
            print("\tROUGE-1 F1 Score:", rouge_scores["rouge-1"]["f"])
            print("\tROUGE-2 F1 Score:", rouge_scores["rouge-2"]["f"])
            print("\tROUGE-L F1 Score:", rouge_scores["rouge-l"]["f"])
            rouge_scoreS.append((rouge_scores["rouge-1"]["f"], rouge_scores["rouge-2"]["f"], rouge_scores["rouge-l"]["f"]))
            bleu_scoreS.append(bleu_score)


        iteration -= 1

    if count > 0:
        rouge_scores1 = 0
        rouge_scores2 = 0
        rouge_scores3 = 0
        bleu_scoreA = 0

        for i in bleu_scoreS:
            bleu_scoreA += i
        for i in rouge_scoreS:
            rouge_scores1 += i[0]
            rouge_scores2 += i[1]
            rouge_scores3 += i[2]

        print("\nAverageBLEU Score:", bleu_scoreA)
        print(f"Average ROUGE Scores for {count} samples")
        print("\tAverage ROUGE-1 F1 Score:", rouge_scores1 / count)
        print("\tAverage ROUGE-2 F1 Score:", rouge_scores2 / count)
        print("\tAverageROUGE-L F1 Score:", rouge_scores3 / count)

    print("\n ---------------------------------------------------------------")


Evaluate(Sample=train_dataset, count=3)  # one sample evaluation


 =================================== Evaluation =================================== 

 ---------------------------------------------------------------
INPUT QUERY:
 How does technology contribute to advancements in animal husbandry?

TARGET RESPONSE:
 Technology in animal husbandry includes innovations like automated feeding systems, precision breeding techniques, and health monitoring devices. These advancements enhance efficiency, reduce costs, and improve overall management practices.

LLM RESPONSE:
 Technology contributes to advancements in animal husbandry by providing tools for data collection and analysis, enhancing productivity and efficiency, and facilitating communication and information sharing.

BLEU Score: 0
ROUGE Scores:
	ROUGE-1 F1 Score: 0.2857142807663474
	ROUGE-2 F1 Score: 0.07843136756632098
	ROUGE-L F1 Score: 0.2857142807663474

 ---------------------------------------------------------------
INPUT QUERY:
 Elaborate on the challenges faced in modern animal husbandr

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

 ## **7. Intergrating  Retreival-Augmented Generation**


"Retrieval-Augmented Generation" is a technique used in natural language processing (NLP) and artificial intelligence (AI) that combines elements of both retrieval-based and generation-based models to improve text generation tasks.

In traditional text generation tasks, such as language modeling or dialogue generation, the model generates responses solely based on the input context without accessing external knowledge. However, retrieval-augmented generation introduces a retrieval step where the model first retrieves relevant information from a large external knowledge source, such as a database or a corpus of documents, before generating the output.

It allows models to leverage external knowledge sources to enhance the quality and relevance of generated text, leading to more accurate and informative outputs.

In [ ]:
!pip install gradient_haystack==0.2.0

  Attempting uninstall: gradient_haystack
    Found existing installation: gradient-haystack 0.4.0
    Uninstalling gradient-haystack-0.4.0:
      Successfully uninstalled gradient-haystack-0.4.0


In [ ]:
from gradient_haystack.embedders.gradient_document_embedder import GradientDocumentEmbedder
from gradient_haystack.embedders.gradient_text_embedder import GradientTextEmbedder
from gradient_haystack.generator.base import GradientGenerator
from haystack import Document, Pipeline
from haystack.components.writers import DocumentWriter
from haystack.document_stores.in_memory.document_store import InMemoryDocumentStore
from haystack.components.retrievers.in_memory.embedding_retriever import InMemoryEmbeddingRetriever
from haystack.components.builders import PromptBuilder
from haystack.components.builders.answer_builder import AnswerBuilder
import os
import requests

In [ ]:
os.environ['GRADIENT_ACCESS_TOKEN'] = "Y2XG6OMr6oZbZETWKkPKogmWshFlD5ft"
os.environ['GRADIENT_WORKSPACE_ID'] = "c571b959-4ce8-474a-b3f3-398c7b347c57_workspace"

fine_tuned_Model_Id = "2e7a849b-239e-44a1-a88a-3be1d77c1f93_model_adapter"

In [ ]:
document_store = InMemoryDocumentStore()
writer = DocumentWriter(document_store=document_store)


document_embedder = GradientDocumentEmbedder(
    access_token=os.environ["GRADIENT_ACCESS_TOKEN"],
    workspace_id=os.environ["GRADIENT_WORKSPACE_ID"],
)

# URL of the online repository where the Raw_Text_Data.txt file is located
url = "https://raw.githubusercontent.com/swafey-karanja/Model-training/main/Raw_Text_Data.txt"

# Send a GET request to download the file
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Read the contents of the downloaded file
    text_data = response.text
else:
    # If the request was not successful, print an error message
    print("Failed to download the file from the URL:", url)

docs = [
    Document(content=text_data)
]

print(len(text_data))

indexing_pipeline = Pipeline()
indexing_pipeline.add_component(instance=document_embedder, name="document_embedder")
indexing_pipeline.add_component(instance=writer, name="writer")
indexing_pipeline.connect("document_embedder", "writer")
indexing_pipeline.run({"document_embedder": {"documents": docs}})

text_embedder = GradientTextEmbedder(
    access_token=os.environ["GRADIENT_ACCESS_TOKEN"],
    workspace_id=os.environ["GRADIENT_WORKSPACE_ID"],
)

generator = GradientGenerator(
    access_token=os.environ["GRADIENT_ACCESS_TOKEN"],
    workspace_id=os.environ["GRADIENT_WORKSPACE_ID"],
    model_adapter_id=fine_tuned_Model_Id,
    max_generated_token_count=350,
)

75697


100%|██████████| 1/1 [00:00<00:00,  2.77it/s]


In [ ]:
prompt = """You are helpful assistant meant to answer questions relating to animal husbandry. Answer the query, based on the
content in the documents. if you dont know the answer respond by saying you are unable to assist with that at the moment.
{{documents}}
Query: {{query}}
\nAnswer:
"""

retriever = InMemoryEmbeddingRetriever(document_store=document_store)
prompt_builder = PromptBuilder(template=prompt)

rag_pipeline = Pipeline()
rag_pipeline.add_component(instance=text_embedder, name="text_embedder")
rag_pipeline.add_component(instance=retriever, name="retriever")
rag_pipeline.add_component(instance=prompt_builder, name="prompt_builder")
rag_pipeline.add_component(instance=generator, name="generator")
rag_pipeline.add_component(instance=AnswerBuilder(), name="answer_builder")
rag_pipeline.connect("generator.replies", "answer_builder.replies")
rag_pipeline.connect("retriever", "answer_builder.documents")
rag_pipeline.connect("text_embedder", "retriever")
rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "generator")


def LLM_Run(question):
    result = rag_pipeline.run(
        {
            "text_embedder": {"text": question},
            "prompt_builder": {"query": question},
            "answer_builder": {"query": question}
        }
    )
    return result["answer_builder"]["answers"][0].data

In [ ]:
Query = "Do bulls show signs of Trichomoniasis?"
print(LLM_Run(Query))

No, bulls do not show signs of Trichomoniasis.


In [ ]:
!pip install anvil-uplink

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45228 sha256=eb23a86c737c3cc6d4f4a99fd9df1f79111d833213d48a13ba3342b4c2010974
  Stored in directory: /root/.cache/pip/wheels/2e/7c/ad/d9c746276bf024d44296340869fcb169f1e5d80fb147351a57
Successfully built ws4py


In [ ]:
import anvil.server

In [ ]:
anvil.server.connect("server_5A33FQHS5NVTR2BFISKONOLE-UOGPLND5WG64J4TQ")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


In [ ]:
@anvil.server.callable
def question_answer(question):
  response = LLM_Run(question)
  print(response)
  return response

In [ ]:
anvil.server.wait_forever()

The issue could be warts or calluses.
Lumps on the skin of cattle can be caused by various factors such as insect bites, skin infections, or tumors. It is essential to monitor the lumps for changes in size, shape, or texture. If the lumps are large, painful, or appear to be growing, it is recommended to seek professional help. A veterinarian can diagnose the cause of the lumps and provide appropriate treatment.
